# Dynamic analysis I
In this notebook, you will learn how to analyze dynamic performance data using the Breathe Design Model api. We make use of the Cycler class to simulate different battery test protocols.

In [30]:
from breathe_design import api_interface as api
from breathe_design import Cycler
from breathe_design import enable_notebook_plotly

enable_notebook_plotly()

In [31]:
base_params = api.get_design_parameters("Molicel P45B")


🔐 No active session. Please sign in to continue.

🚀 Open this URL in a browser and verify the code shown is MCGG-STSJ
🔗 https://breathe-platform.uk.auth0.com/activate?user_code=MCGG-STSJ

⏳ Waiting for user authorization...

✅ Authentication successful!


We will first obtain the baseline parameters for the Molicel P45B cell. Thereafter, we will create a virtual cell by reducing the NP ratio by 5%. We will then compare the equilibrium KPIs as well as the temperature and voltage responses in dynamic simulations. We will base the C-Rate on the baseline cell capacity.

In [32]:
results = api.get_eqm_kpis("Molicel P45B")
results.capacity

np.float64(4.499999999999997)

In [33]:
designs = [{"designName": "Lower NP", "NPratio": base_params["NPratio"] * 0.95}]

We instantiate a Cycler object with the selected unit (C) and the baseline cell capacity. Now all subsequent cycler inputs will use C-rate for current control.

In [34]:
cycler = Cycler(selected_unit="C", cell_capacity=results.capacity)

Here we can inspect the CCCV input parameters and type of the Cycler object.

In [35]:
? cycler.cccv

Signature:
 cycler.cccv(
    I_chg: float,
    I_dch: float,
    I_cut: float,
    V_max: float,
    V_min: float,
    t_restC: float,
    t_restD: float,
) -> dict
Docstring:
Create a Constant Current Constant Voltage (CCCV) cycler protocol.

Parameters:
    I_chg (float): The charging current in [A || C]. Required.
    I_dch (float): The discharging current in [A || C]. Required.
    I_cut (float): The cut-off current in [A || C]. Required.
    V_max (float): The maximum charging voltage in V. Required.
    V_min (float): The minimum discharging voltage in V. Required.
    t_restC (float): The rest time (in seconds) after charging. Required.
    t_restD (float): The rest time (in seconds) after discharging. Required.

Returns:
    (dict): representing the CCCV cycling protocol.
File:      d:\git\breathe_design\breathe_design\cycler.py
Type:      method

In [36]:
cycler_input = cycler.cccv(1.0, -1.0, 0.01, 4.2, 2.6, 60.0, 60.0)

In [37]:
cycler_input

{'cycle_type': 'CCCV',
 'selected_unit': 'C',
 'control_parameters': {'I_chg': 1.0,
  'I_dch': -1.0,
  'I_cut': 0.01,
  'V_max': 4.2,
  'V_min': 2.6,
  't_restC': 60.0,
  't_restD': 60.0}}

In [38]:
# run a single simulation
output = api.run_sim(
    base_battery="Molicel P45B",
    cycler=cycler_input,
    designs=designs,
    initialSoC=0.5,
    initialTemperature_degC=21.0,
)

The dynamic simulation is executed for each design and can be plotted using the `plot_voltage_response()` function.

In [39]:
output.plot_voltage_response()

You can also explore the dynamic variables and print them using the `print_dynamic_variables()` and therafter call them with`plot_dynamic_response()` method.

In [40]:
output.print_dynamic_variables()

Available variables:
Time
voltModel
voltAnode
voltCathode
ocvModel
ocvAnode
ocvCathode
tempSurfaceModel
socModel
socAnode
socCathode
currModel
heatFluxModel


In [41]:
output.plot_dynamic_response("tempSurfaceModel")